# How to Filter Bad Data in Instruction Tuning Datasets

This notebook demonstrates how to use Cleanlab TLM to detect low-quality prompt-response datapoints in a fine-tuning dataset. 

## Import Dependencies

In [ ]:
!pip install cleanlab_studio pandas tqdm

In [30]:
from cleanlab_studio import Studio
import pandas as pd
from tqdm import tqdm

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

## Initialize TLM using Cleanlab Studio API Key

Replace with your own key. If you do not yet have an account, sign up for a free one here: https://app.cleanlab.ai/

In [31]:
api_key = 'e8509775a2cd4f5eb1195c7441dd530e'
batch_size = 64
tlm = Studio(api_key).TLM(quality_preset="medium", max_concurrent_requests=batch_size)

## Fetching Data

Download the dataset here: https://huggingface.co/datasets/databricks/databricks-dolly-15k

The fine-tuning dataset contains 15000 instruction-response pairings covering 7 categories. Three of these categories (open q&a, classification, creative writing, brainstorming) do not incorporate context in their input while the remaining four categories (closed q&a, information extraction, summarization, brainstorming) do.

In [32]:
df = pd.read_json("databricks-dolly-15k.jsonl", lines=True)
df.head(5)

,instruction,context,response,category
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.","Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.",closed_qa
1,Which is a species of fish? Tope or Rope,,Tope,classification
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.,open_qa
3,"Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?",,The name of the third daughter is Alice,open_qa
4,When was Tomoaki Komorida born?,"Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer. In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played as a regular player and the club was promoted to J2 in 2008. Although he did not play as much, he still played in many matches. In 2010, he moved to Indonesia and joined Persela Lamongan. In July 2010, he returned to Japan and joined the J2 club Giravanz Kitakyushu. He played often as a defensive midfielder and center back until 2012 when he retired.","Tomoaki Komorida was born on July 10,1981.",closed_qa


## Obtaining Scores on No-Context Datapoints
In the following example, we will filter for ```open_qa``` datapoints. This category contains direct questions and answers regarding general world knowledge. For the sake of this tutorial, we will work with a subset of the dataset. After you are familiar with TLM by the end of this tutorial, we encourage you to investigate the rest of the dataset on your own, or any other dataset you are interested in!

In [36]:
open_qa_df = df[df['category'] == 'open_qa'].copy(deep=True)
open_qa_df = open_qa_df[open_qa_df['context'].str.len() == 0]
open_qa_df = open_qa_df.drop(columns=['context'])
open_qa_df = open_qa_df.sample(n=600, random_state=42)

open_qa_df.head(5)

,instruction,response,category
13826,What is a 'Public' School in the UK?,"Public School is a term used for an independent fee-paying school in the UK. This is because when the schools were initially set-up, they were for the use of fee-paying parents who would share a teacher as a replacement for a dedicated at-home tutor for their children",open_qa
4315,What are some of the causes for the 1929 US stock market crash?,"The 1920’s were a booming time for Americans as the stock market was surging due to speculative investing. Stock prices rose to records heights which drove the public to invest their liquid assets into securities. Billions of dollars were drawn from banks for broker’s loans to carry margin accounts. People were mortgaging their homes to pour money into stocks. In late 1929, the market eventually overheated and prices began to fall despite efforts by the banks to buy up large blocks of stock to stem the losses. Eventually, the panic was too much and the market crashed on Black Monday, October 24th when the market closed down 12.8 percent. The sharp decline in the 1929 stock market eventually led to the Great Depression of the 1930’s.",open_qa
1409,Why do people live in homes?,"People live in homes because its a way to protect you from the elements of nature like storms, heavy rain, lightning, heat and humidity. Its a place that offers comfort with air conditioning, offers a comfortable place to sleep, and offers a comfortable place to cook and eat and have entertaining activities like television, and spending time with your family.",open_qa
7931,What are the best places to eat and top Japanese sushi restaurant in San Mateo?,The best Japanese Restaurant in San Mateo are Sushi Sam and Sushi Maruyama,open_qa
9273,"Give me a title of one of the ""Harry Potter"" novels.",Harry Potter and the Half-Blood Prince,open_qa


We leverage ```batch_get_confidence_score``` to efficiently obtain the confidence scores. Each call can process up to a batch of 64 input-output pairs at once, and takes around a minute to execute. The resulting scores will be saved in our ```open_qa_results``` dataframe.

In [38]:
idx_start = 0
open_qa_results = open_qa_df.copy(deep=True) 
open_qa_results["confidence_score"] = None

for i in tqdm(range(0, len(open_qa_results), batch_size)):
    end_index = min(i + batch_size, len(open_qa_results))
    
    try:
        batch_instructions = open_qa_results.iloc[i:end_index]["instruction"].tolist()
        batch_responses = open_qa_results.iloc[i:end_index]["response"].tolist()
    
        batch_scores = tlm.batch_get_confidence_score(
            batch_instructions, batch_responses, retries=1
        )
        
        batch_scores = list(batch_scores)
        print("Updating rows:", i, "to", end_index-1)
        open_qa_results.iloc[i:end_index, open_qa_results.columns.get_loc("confidence_score")] = batch_scores
    
    except Exception as error:
        print("Failed at index ", i, " Error: ", str(error))

 10%|████▏                                     | 1/10 [00:42<06:26, 42.93s/it]

Updating rows: 0 to 63


 20%|████████▍                                 | 2/10 [01:22<05:26, 40.79s/it]

Updating rows: 64 to 127


 30%|████████████▌                             | 3/10 [02:00<04:36, 39.43s/it]

...

Updating rows: 448 to 511


 90%|█████████████████████████████████████▊    | 9/10 [05:40<00:37, 37.83s/it]

Updating rows: 512 to 575


100%|█████████████████████████████████████████| 10/10 [05:54<00:00, 35.46s/it]

Updating rows: 576 to 599


Now that TLM has finished running, let's inspect our low-confidence datapoints:

In [39]:
open_qa_results = open_qa_results.sort_values(by="confidence_score")
# open_qa_results.to_csv('open_qa_results.csv', index=False)
open_qa_results.head(10)

,instruction,response,category,confidence_score
3235,What did Stevie Ray Vaughan name his most beloved guitar?,Jimbo.,open_qa,0.002569
6265,"In the series A Song of Ice and Fire, who is the founder of House Casterly?","Corlos, son of Caster",open_qa,0.015284
14531,"In the series A Song of Ice and Fire, who is the founder of House Tyrell?",Ser Alester Tyrell,open_qa,0.016146
14039,Name a famous American author who wrote stories about the American South during the early 20th century.,"Erskine Caldwell is a celebrated American author who wrote short stories on the rural deep south in the United States of America. Most of his work was published in the 1930’s through the 1950’s with a few being Tobacco Road, Georgia Boy, and God’s Little acre.",open_qa,0.060823
7182,How many Anglo-Dutch wars were there?,There were six Anglo-Dutch wars between 1585 and 1604.,open_qa,0.061104
8267,Which NFL team has won every Super Bowl they played in?,"Tampa Bay Buccaneers, Baltimore Ravens, New Orleans Saints, New York Jets",open_qa,0.062069
10777,Which artist's entire 1985 Live Aid set consisted of just a single cover song?,Elvis Costello.,open_qa,0.064325
7069,FTX Crypto Cup Chess was played at,Miami,open_qa,0.071002
2641,Panchatantra stories were written by,"Panchatantra stories were written by Kautilya in Sanskrit. The stories are about animals and humans living together in harmony. They teach valuable lessons about justice, compassion, and loyalty.",open_qa,0.076307
4449,"In the series A Song of Ice and Fire, who is the founder of House Towers?",Walton Towers,open_qa,0.082063


At this point, we would manually verify each datapoint, and choose to either correct or remove the low-quality ones.

## Obtaining Scores on Datapoints with Context
In the following example, we filter for ```closed_qa``` datapoints. This category contains questions that can be answered using only the information contained in the given context. For datapoints that incoporate context, we can simply prepend the context text to the instruction as follows. The rest of the steps are the same as the previous example.

In [42]:
closed_qa_df = df[df['category'] == 'closed_qa'].copy(deep=True)
closed_qa_df['instruction'] = 'context: ' + closed_qa_df['context'].astype(str) + '\nquestion: ' + closed_qa_df['instruction'].astype(str)
closed_qa_df = closed_qa_df.drop(columns=['context'])
closed_qa_df = closed_qa_df.sample(n=600, random_state=42)

closed_qa_df.head(5)

,instruction,response,category
10117,"context: Douglas Stuart Moore (August 10, 1893 – July 25, 1969) was an American composer, songwriter, organist, pianist, conductor, educator, actor, and author. A composer who mainly wrote works with an American subject, his music is generally characterized by lyricism in a popular or conservative style which generally eschewed the more experimental progressive trends of musical modernism. Composer Virgil Thomson described Moore as a neoromantic composer who was influenced by American folk music. While several of his works enjoyed popularity during his lifetime, only his folk opera The Ballad of Baby Doe (1956) has remained well known into the 21st century.\nquestion: What is Douglas Stuart Moore's best remembered for?",The Ballad of Baby Doe (1956) as it has remained well known into the 21st century.,closed_qa
10001,"context: Oak Island has been a subject for treasure hunters ever since the late 1700s, with rumours that Captain Kidd's treasure was buried there. While there is little evidence to support what went on during the early excavations, stories began to be published and documented as early as 1856. Since that time there have been many theories that extend beyond that of Captain Kidd which include among others religious artifacts, manuscripts, and Marie Antoinette's jewels. The ""treasure"" has also been prone to criticism by those who have dismissed search areas as natural phenomena.\nquestion: What is hidden at Oak Island?","Oak Island is rumored to have Captain Kidd's treasure buried. However despite numerous excavation attempts over the past 100 years nothing has been discovered. There are other theories which claim religious artifacts, manuscripts, or Marie Antionette's jewels are hidden at Oak Island.",closed_qa
6734,"context: Belize, formerly known as British Honduras, is a Caribbean country located on the northeastern coast of Central America. Belize is bordered on the northwest by Mexico, on the east by the Caribbean Sea, and on the south and west by Guatemala. It has an area of 22,970 square kilometres (8,867 sq mi) and a population of 408,487 (2019). Its mainland is about 290 km (180 mi) long and 110 km (68 mi) wide. It has the lowest population and population density in Central America.\nquestion: Given this short paragraph about Belize, how many people live there and what is notable about its population density?","Belize has a population of 408,487 people and has the lowest population density in Central America.",closed_qa
13939,"context: In 1950s Japan, university professor Tatsuo Kusakabe and his daughters Satsuki and Mei (approximately ten and four years old, respectively) move into an old house close to the hospital where the girls' mother Yasuko is recovering from a long-term illness. The house is inhabited by small, dark, dust-like house spirits called susuwatari, which can be seen when moving from bright places to dark ones.[note 1] When the girls become comfortable in their new house, the susuwatari leave to find another empty house. Mei later discovers two small spirits who lead her into the hollow of a large camphor tree. She befriends a larger spirit, which identifies itself using a series of roars she interprets as ""Totoro"". Mei thinks Totoro is the troll from her illustrated book Three Billy Goats Gruff, with her mispronouncing troll. Mei falls asleep atop Totoro but when Satsuki finds her, she is on the ground. Despite many attempts, Mei cannot show her family Totoro's tree. Tatsuo comforts her by telling her Totoro will reveal himself when he wants to.\nquestion: Who are the little Japanese girls who find Totoro?",Mei and Satsuki are the two Japanese girls who discover Totoro in the famous Japanese anime My Neighbour Totoro.,closed_qa
9779,"context: The Bakersfield Majestics, formerly the Bakersfield Magic, are an American basketball team based in Bakersfield, California, and members in the minor professional The Basketball League (TBL).\n\nStarting Th

In [ ]:
idx_start = 0
closed_qa_results = closed_qa_df.copy(deep=True) 
closed_qa_results["confidence_score"] = None

for i in tqdm(range(0, len(closed_qa_results), batch_size)):
    end_index = min(i + batch_size, len(closed_qa_results))
    try:
        batch_instructions = closed_qa_results.iloc[i:end_index]["instruction"].tolist()
        batch_responses = closed_qa_results.iloc[i:end_index]["response"].tolist()
    
        batch_scores = tlm.batch_get_confidence_score(
            batch_instructions, batch_responses, retries=1
        )
        
        batch_scores = list(batch_scores)
        closed_qa_results.iloc[i:end_index, closed_qa_results.columns.get_loc("confidence_score")] = batch_scores
        
    except Exception as error:
        print("Failed at index ", i, " Error: ", str(error))

In [44]:
closed_qa_results = closed_qa_results.sort_values(by="confidence_score")
# closed_qa_results.to_csv('closed_qa_results.csv', index=False)
closed_qa_results.head(10)

,instruction,response,category,confidence_score
6770,"context: In 2011 Kathy Ryan, director of photography for The New York Times Magazine commissioned Prager to shoot 12, 1 minute films with some film actors from that year, inspired by ""cinematic villainy"". Prager won a News and Documentary Emmy Award for New Approaches to News & Documentary Programming: Arts, Lifestyle & Culture for her Touch of Evil short films.\nquestion: What magazine was Alex Prager the director of Photography for?",Alex Prager was the director of photography for the New York Times Magazine.,closed_qa,0.064139
9452,"context: The relative size of a Foley catheter is described using French units (F). Alternatively, the size of a 10 F catheter might be expressed as 10 Ch (Charriere units – named after a 19th century French scientific instrument maker, Joseph-Frédéric-Benoît Charrière). The most common sizes are 10 F to 28 F. 1 F is equivalent to 0.33 mm = .013"" = 1/77"" of diameter. Foley catheters are usually color coded by size with a solid color band at the external end of the balloon inflation tube, allowing for easy identification of the size. Note: Colors for French sizes 5, 6, 8, 10 may vary significantly if intended for pediatric patients. Color for French size 26 may also be pink instead of black.\nhttps://en.wikipedia.org/wiki/Foley_catheter\nquestion: Based on this paragraph, what is the diameter of a 10 F Foley catheter in mm?","According to this paragraph, the diameter of a 1 F catheter is 1/77mm so the diameter of a 10 F catheter is 10/77mm.",closed_qa,0.113954
956,"context: The dead man zone is the area directly around a bushfire that is likely to burn within five minutes given the current wind conditions or an anticipated change in wind direction. The distance this zone extends from the firefront is highly dependent on terrain, windspeed, fuel type and composition, relative humidity and ambient temperature, and can range from under 100 metres (330 ft) to well over 1 kilometre (3,300 ft).\nquestion: Based on this article, considerations of what time zone is used for determining what land is in the dead man zone?",A five minute period in the future is used for creating the dead man zone.,closed_qa,0.142538
9953,"context: Charles André Weiss (September 30, 1858 in Mulhouse - August 31, 1928 in the Hague) was a French jurist. He was professor at the Universities of Dijon and Paris and served from 1922 until his death as judge of the Permanent Court of International Justice.\n\nLife\nAndré Weiss was born in Mulhouse in 1858 and completed a degree in law at the University of Paris in 1880. The following year he became a professor at the University of Dijon. In 1891 he moved to the Law School of the University of Paris. There he was from 1896 to 1908 a full professor of civil law, and from 1908 he held the chair for international law and private international law. From 1907 he also acted as legal advisor to the French Ministry of Foreign Affairs. He was a close associate of Secretary of State Aristide Briand and participated as a delegate to the Paris Peace Conference at the end of World War I. He led one of the subcommissions of the conference.\n\nFrom 1920, he was a member of the Permanent Court of Arbitration in The Hague. In September 1921, he was elected by the Assembly and the Council of the League of Nations to the post of judge on the newly formed Permanent International Court. His fellow judges elected him vice president at the beginning of the Tribunal's work on February 3, 1922. He was confirmed in this office in 1924 and 1927. He died before the end of his term as Judge and Vice President in August 1928 in the Hague. His compatriot, Henri Fromageot, was elected his successor, and in the office of vice-president he was succeeded by the Swiss Max Huber.\n\nAndré Weiss taught in 1923 as a lecturer at the Hague Academy of International Law. He was a member of the Institut de Droit International from 1887, acting as president of its 30th 